# Quick Start using lists and dictionaries
[![pandasnative.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/quick-start/pandasnative.ipynb)](https://colab.ampl.com)

Description: Quick Start using lists and dictionaries to load and retrieve data

Tags: amplpy, quick-start, highlights

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

In [19]:
# Install dependencies
!pip install -q amplpy pandas numpy

In [20]:
# Google Colab & Kaggle integration
MODULES, LICENSE_UUID = ['highs'], None
from amplpy import tools
ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, globals_=globals()) # instantiate AMPL object and register magics

# Diet Model

Consider the problem of choosing prepared foods to meet certain nutritional requirements.

Sets:
- `NUTR`: set of nutrients to consider
- `FOOD`: set of food to consider

Parameters:
- `cost {FOOD}`: cost of each food
- `f_min {FOOD}`: minimum amount of food to buy
- `f_max {FOOD}`: maximum amount of food to buy
- `n_min {NUTR}`: minimum amount required of each nutrient
- `n_max {NUTR}`: maximum amount allowed of each nutrient
- `amt {NUTR, FOOD}`: amount of each nutrient in each food

Variables:
- `Buy {FOOD}`: amount of food to buy

Objective:
- `Total_Cost`: total cost of the diet

Constraints:
- `Diet {NUTR}`: ensure that the nutritional requirements are satisfied by the diet. 

The problem is then modeled as follows:

In [21]:
%%ampl_eval
reset;
set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

## Load the data using lists and dictionaries

In [22]:
foods = ["BEEF", "CHK", "FISH", "HAM", "MCH", "MTL", "SPG", "TUR"]
nutrients = ["A", "C", "B1", "B2", "NA", "CAL"]
ampl.set["FOOD"] = foods
ampl.param["cost"] = [3.59, 2.59, 2.29, 2.89, 1.89, 1.99, 1.99, 2.49]
ampl.param["f_min"] = [2, 2, 2, 2, 2, 2, 2, 2]
ampl.param["f_max"] = [10, 10, 10, 10, 10, 10, 10, 10]
ampl.set["NUTR"] = nutrients
ampl.param["n_min"] = [700, 700, 700, 700, 0, 16000]
ampl.param["n_max"] = [20000, 20000, 20000, 20000, 50000, 24000]
amounts = [
    [60, 8, 8, 40, 15, 70, 25, 60],
    [20, 0, 10, 40, 35, 30, 50, 20],
    [10, 20, 15, 35, 15, 15, 25, 15],
    [15, 20, 10, 10, 15, 15, 15, 10],
    [928, 2180, 945, 278, 1182, 896, 1329, 1397],
    [295, 770, 440, 430, 315, 400, 379, 450],
]
ampl.param["amt"] = {
    (nutrient, food): amounts[i][j]
    for i, nutrient in enumerate(nutrients)
    for j, food in enumerate(foods)
}

# Solve with HiGHS

In [23]:
# Specify the solver to use (e.g., HiGHS)
ampl.option["solver"] = "highs"
# Solve
ampl.solve()
assert ampl.get_value("solve_result") == "solved"
# Get objective entity by AMPL name
totalcost = ampl.get_objective('Total_Cost')
# Print it
print("Objective is:", totalcost.value())

HiGHS 1.4.0: HiGHS 1.4.0: optimal solution; objective 119.9897589
4 simplex iterations
0 barrier iterations
absmipgap=119.99, relmipgap=inf
Objective is: 119.98975893599335


## Reassign data for specific instances and resolve

In [24]:
# Reassign data - specific instances
cost = ampl.get_parameter('cost')
cost.set_values({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# Resolve and display objective
ampl.solve()
assert ampl.get_value("solve_result") == "solved"
print("New objective value:", totalcost.value())

Increased costs of beef and ham.
HiGHS 1.4.0: HiGHS 1.4.0: optimal solution; objective 144.0120033
0 simplex iterations
0 barrier iterations
absmipgap=144.012, relmipgap=inf
New objective value: 144.01200332502077


### Reassign data for all instances and resolve

In [25]:
# Reassign data - all instances
cost.set_values([3, 5, 5, 6, 1, 2, 5.01, 4.55])
print("Updated all costs.")

# Resolve and display objective
ampl.solve()
assert ampl.get_value("solve_result") == "solved"
print("New objective value:", totalcost.value())

Updated all costs.
HiGHS 1.4.0: HiGHS 1.4.0: optimal solution; objective 164.10625
4 simplex iterations
0 barrier iterations
absmipgap=164.106, relmipgap=inf
New objective value: 164.10625


## Get the values of the variable Buy in a pandas.DataFrame object

In [26]:
df = ampl.get_variable('Buy').get_values().to_pandas()
df

,Buy.val
BEEF,10.000000
CHK,2.000000
FISH,2.000000
HAM,6.851974
MCH,10.000000
MTL,10.000000
SPG,6.765351
TUR,2.000000


## Get the values of an expression into a pandas.DataFrame object

In [27]:
df = ampl.get_data('{j in FOOD} 100*Buy[j]/Buy[j].ub').to_pandas()
df

,100*Buy[j]/(Buy[j].ub)
BEEF,100.000000
CHK,20.000000
FISH,20.000000
HAM,68.519737
MCH,100.000000
MTL,100.000000
SPG,67.653509
TUR,20.000000
